In [ ]:
import os
import json
import requests
from dotenv import load_dotenv
load_dotenv()

API_URL = os.environ.get('API_BASE_URL', 'http://super-rag:8081')

PINECONE_INDEX = os.getenv("PINECONE_INDEX", "")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY", "")
PINECONE_HOST = os.getenv("PINECONE_HOST", "")

WEAVIATE_API_KEY = os.environ.get('WEAVIATE_API_KEY', '')
WEAVIATE_INDEX = os.environ.get('WEAVIATE_INDEX', '')
WEAVIATE_URL = os.environ.get('WEAVIATE_URL', '')

CONFLUENCE_API_TOKEN=os.environ.get('CONFLUENCE_API_TOKEN', '')
CONFLUENCE_USER_EMAIL=os.environ.get('CONFLUENCE_USER_EMAIL', '')
CONFLUENCE_URL=os.environ.get('CONFLUENCE_URL', '')

print("API_URL:", API_URL)

# print("PINECONE_API_KEY:", PINECONE_API_KEY)
print("PINECONE_INDEX:", PINECONE_INDEX)
print("PINECONE_HOST:", PINECONE_HOST)

# print("WEAVIATE_API_KEY:", WEAVIATE_API_KEY)
print("WEAVIATE_INDEX:", WEAVIATE_INDEX)
print("WEAVIATE_URL:", WEAVIATE_URL)

# print("CONFLUENCE_API_TOKEN:", CONFLUENCE_API_TOKEN)
print("CONFLUENCE_USER_EMAIL:", CONFLUENCE_USER_EMAIL)
print("CONFLUENCE_URL:", CONFLUENCE_URL)

In [ ]:
# Ingest a file
url = f"{API_URL}/api/v1/ingest"
    
payload = {
    "confluence": {
        "connector_config": {
            "access_config": {
                "api_token": CONFLUENCE_API_TOKEN
            },
            "user_email": CONFLUENCE_USER_EMAIL,
            "url": CONFLUENCE_URL,
            "max_num_of_spaces": 500,
            "max_num_of_docs_from_each_space": 100,
            "spaces": ["KB", "SD"]
        }
    },
    "vector_database": {
        "type": "weaviate",
        "config": {
            "api_key": WEAVIATE_API_KEY,
            "host": WEAVIATE_URL
        }
    },
    "index_name": "ConfluenceTest12",
    "encoder": {
        "type": "cohere",
        "name": "embed-english-v3.0",
        "dimensions": 1024,
    },
}

response = requests.post(url, json=payload)

if response.status_code == 200:
    try:
        pretty_json = json.dumps(response.json(), indent=4, ensure_ascii=False)
        print(pretty_json)  
    except ValueError:
        print("Response content is not valid JSON.")
else:
    print(f"Request failed with status code {response.status_code}: {response.text}")

In [ ]:
# Query the index
query_url = f"{API_URL}/api/v1/query"

query_payload = {
    "input": "How do I reset my password?",
    "vector_database": {
        "type": "weaviate",
        "config": {
            "api_key": WEAVIATE_API_KEY,
            "host": WEAVIATE_URL,
        }
    },
    "index_name": "ConfluenceTest12",
    "encoder": {
        "type": "cohere",
        "name": "embed-english-v3.0",
        "dimensions": 1024,
    }
}

query_response = requests.post(query_url, json=query_payload)

pretty_json = json.dumps(query_response.json(), indent=4, ensure_ascii=False)
print(pretty_json)

In [ ]:
# Delete the index
query_url = f"{API_URL}/api/v1/delete"

delete_payload = {
    "file_url": "https://arxiv.org/pdf/2210.03629.pdf",
    "vector_database": {
        "type": "pinecone",
        "config": {
            "api_key": WEAVIATE_API_KEY,
            "host": WEAVIATE_URL,
        }
    },
    "index_name": "ConfluenceTest",
    "encoder": {
        "type": "cohere",
        "name": "embed-english-v3.0",
        "dimensions": 1024,
    }
}

delete_response = requests.delete(query_url, json=delete_payload)

pretty_json = json.dumps(delete_response.json(), indent=4, ensure_ascii=False)
print(pretty_json)